In [ ]:
%matplotlib notebook
import numpy as np
import time
import tables as tb
import matplotlib.pyplot as plt

from tjmonopix.tjmonopix import TJMonoPix
from tjmonopix.online_monitor import noise_monitor, plots
from tjmonopix.analysis import analysis_functions

import tjmonopix.scans.injection_scan

chip = TJMonoPix(conf="../tjmonopix/tjmonopix_mio3.yaml", no_power_reset=False)
chip.init(fl="EN_HV")

chip['data_rx'].CONF_START_FREEZE = 64 #default 3
chip['data_rx'].CONF_STOP_FREEZE = 100 #default 40
chip['data_rx'].CONF_START_READ = 66 #default 6
chip['data_rx'].CONF_STOP_READ = 68 #default 7
chip['data_rx'].CONF_STOP = 105 #default 45

In [ ]:
chip.get_power_status()

In [ ]:
vl_dac = 40
vh_dac = 80
vreset_dac = 35
icasn_dac = 0  # This sets the baseline voltage of the discriminator input
ireset_dac = 2
ithr_dac = 5  # Also this sets the baseline voltage of the discriminator input
idb_dac = 20  # This sets the threshold of the discriminator. It is changed later.
ibias_dac = 100

vl = chip.set_vl_dacunits(vl_dac,1)
vh = chip.set_vh_dacunits(vh_dac,1)

chip.write_conf()

vreset = chip.set_vreset_dacunits(vreset_dac, 1) 
icasn = chip.set_icasn_dacunits(icasn_dac,1) 

ireset = chip.set_ireset_dacunits(ireset_dac,1,1) 
ithr = chip.set_ithr_dacunits(ithr_dac,1) 
idb = chip.set_idb_dacunits(idb_dac,1) 
ibias = chip.set_ibias_dacunits(ibias_dac,1) 

chip.write_conf()

In [ ]:
chip.unmask_all()
plots.tj_plot(chip, dt = 0.2, wait_inj=False);


In [ ]:
hits, _, _ = chip.recv_data_summary(delta_t)
with tb.open_file('test.h5', "w") as f:
    description = np.zeros((1,), dtype=hits.dtype)
    hit_table = f.create_table(f.root, name="Hits", description=description, title="Hit data")
    hit_table.append(hits)
    hit_table.flush()